<a href="https://colab.research.google.com/github/goya5858/commonlitreadabilityprize/blob/main/working/RoBERTa_infference.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Install libs

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
compe_name = 'commonlitreadabilityprize'
%cd /content/drive/MyDrive/kaggle/works/$compe_name/working/

!pip install -q -q -q -U albumentations
!pip install -q -q -q -U torch
!pip install -q -q -q timm
!pip install -q -q -q pytorch_lightning
!pip install -q -q -q -U transformers
!pip install -q -q -q -U sentencepiece

import os
import re
import gc
import sys
import time
import copy
import random
import warnings
from tqdm import tqdm_notebook as tqdm

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import cv2
import PIL.Image

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

import albumentations
from albumentations.pytorch.transforms import ToTensorV2

import torch.optim as optim
from torch.optim import lr_scheduler
from torch.optim.lr_scheduler import CosineAnnealingLR

import timm

import nltk
from wordcloud import WordCloud
nltk.download('stopwords')
from nltk.corpus import stopwords
stop=stopwords.words('english')
from nltk.stem import WordNetLemmatizer
from textblob import TextBlob,Word
from collections import Counter
import string
from torch.nn.utils.rnn import pad_sequence

from torch.nn import MSELoss

from torch.cuda import amp

from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn import model_selection

import transformers
from transformers import get_linear_schedule_with_warmup, AdamW

from transformers import *

def seed_everything(seed=1234):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    #tf.random.set_seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True

/content/drive/MyDrive/kaggle/works/commonlitreadabilityprize/working
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


# Dataset

In [3]:
DEVICE = 'cuda:0'

get_tokenizer = RobertaTokenizer
#get_model     = BartForSequenceClassification
#get_tokenizer = AutoTokenizer #RobertaTokenizer より汎用性ある気がする　使い勝手的な面で
#get_model     = RobertaForSequenceClassification
get_model     = RobertaModel

#VOCAB_PATH = 'roberta-base'
VOCAB_PATH = '../input/models/RoBERTa_model'
MODEL_PATH = '../input/models/RoBERTa_model'
#VOCAB_PATH = 'bert-base-uncased'
#MODEL_PATH = 'bert-base-uncased'

In [4]:
ROOT = f"../input/{compe_name}/"
MODEL_ROOT = "../input/models/"

df = pd.read_csv(ROOT+'train.csv')
df.head()

,id,url_legal,license,excerpt,target,standard_error
0,c12129c31,NaN,NaN,When the young people returned to the ballroom...,-0.340259,0.464009
1,85aa80a4c,NaN,NaN,"All through dinner time, Mrs. Fayre was somewh...",-0.315372,0.480805
2,b69ac6792,NaN,NaN,"As Roger had predicted, the snow departed as q...",-0.580118,0.476676
3,dd1000b26,NaN,NaN,And outside before the palace a great garden w...,-1.054013,0.450007
4,37c1b32fb,NaN,NaN,Once upon a time there were Three Bears who li...,0.247197,0.510845


In [5]:
test_df = pd.read_csv(ROOT+'test.csv')
test_df.head()

,id,url_legal,license,excerpt
0,c0f722661,NaN,NaN,My hope lay in Jack's promise that he would ke...
1,f0953f0a5,NaN,NaN,Dotty continued to go to Mrs. Gray's every nig...
2,0df072751,NaN,NaN,It was a bright and cheerful scene that greete...
3,04caf4e0c,https://en.wikipedia.org/wiki/Cell_division,CC BY-SA 3.0,Cell division is the process by which a parent...
4,0e63f8bea,https://en.wikipedia.org/wiki/Debugging,CC BY-SA 3.0,Debugging is the process of finding and resolv...


In [6]:
def prep_text(text_df):
    text_df = text_df.str.replace("\n","",regex=False)
    return text_df.str.replace("\'s",r"s",regex=True).values

df['excerpt']      = prep_text(df['excerpt'])
test_df['excerpt'] = prep_text(test_df['excerpt'])

MAX_SEQUENCE_LENGTH = df['excerpt'].apply(lambda x: len(x.split())).max()

tokenizer = get_tokenizer.from_pretrained(VOCAB_PATH,
                                          model_max_length=MAX_SEQUENCE_LENGTH
                                          )
df['token']          = df['excerpt'].apply(tokenizer)
test_df['token']   = test_df['excerpt'].apply(tokenizer)

Token indices sequence length is longer than the specified maximum sequence length for this model (222 > 205). Running this sequence through the model will result in indexing errors


In [7]:
class CLPDataset(Dataset):
    def __init__(self, df):
        super().__init__()
        self.token  = df.token

    def __len__(self):
        return self.token.shape[0]
  
    def __getitem__(self, idx):
        if len(self.token.iloc[0]) == 2:
            return (
                    torch.tensor( self.token.iloc[idx].input_ids ), 
                    torch.tensor( self.token.iloc[idx].attention_mask )
                  )
        if len(self.token.iloc[idx]) == 3:
            return (
                    torch.tensor(self.token.iloc[idx].input_ids), \
                    torch.tensor(self.token.iloc[idx].token_type_ids), \
                    torch.tensor(self.token.iloc[idx].attention_mask)
                    )

In [8]:
def collate_fn(batch):
    inputs = batch
    try:
        ids, types, masks = zip(*inputs)
        ids   = pad_sequence(ids, batch_first=True).to(DEVICE)
        types = pad_sequence(types, batch_first=True).to(DEVICE)
        masks = pad_sequence(masks, batch_first=True).to(DEVICE)
        return {
                    "input_ids"      : ids, \
                    "token_type_ids" : types, \
                    "attention_mask" : masks
                  }

    except ValueError:
        ids, masks = zip(*inputs)
        ids   = pad_sequence(ids, batch_first=True).to(DEVICE)
        masks = pad_sequence(masks, batch_first=True).to(DEVICE)
        return {
                    "input_ids"      : ids, \
                    "attention_mask" : masks
                  }

In [9]:
def get_testloader(df, BATCH_SIZE):
    test_dataset = CLPDataset(df=df)
    test_loader = DataLoader(
        dataset = test_dataset,
        batch_size = BATCH_SIZE,
        shuffle = False,
        collate_fn=collate_fn,
        #num_workers = -1
        )
    return test_loader

# model

In [10]:
sample_loader = get_testloader(test_df, BATCH_SIZE=1)
sample_data = iter(sample_loader).next()
sample_data

{'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
          1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
          1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
          1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
          1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
          1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
          1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
          1, 1]], device='cuda:0'),
 'input_ids': tensor([[    0,  2387,  1034,  4477,    11,  2722,    29,  4198,    14,    37,
             74,   489,    10,  4520,  1109,  6574,    11,     5,  2853,   527,
              7,  4704,   162,    15,   127,   768,     4,   374,    10,   699,
            363,    42,  1109,    21,  7097,    31,     5,  3375,     6,    53,
           7421,    50,    97,    38,  

In [11]:
class CLPmodel(nn.Module):
    def __init__(self, check_size=False):
        super().__init__()
        self.model  = get_model.from_pretrained(MODEL_PATH).to(DEVICE)
        #self.model  = get_model.from_pretrained('roberta-base').to(DEVICE)
        #self.model  = RobertaForSequenceClassification.to(DEVICE)
        OUTPUT_SIZE = self.model(**sample_data)[0].shape[-1]
        self.drop = nn.Dropout(0.5)
        self.fc = nn.Linear(in_features=OUTPUT_SIZE, out_features=1)
        
        if check_size:
            print('base_model`s output_size :', OUTPUT_SIZE)
            print(DEVICE)
  
    def forward(self,inputs):
        out = self.model(**inputs)
        last_hiddens = out[0]
        out = self.drop(last_hiddens[:,0,:].squeeze(1))
        return self.fc(out)

In [12]:
model = CLPmodel(check_size=True)
del model, _
gc.collect()

base_model`s output_size : 768
cuda:0


415

# Infference

In [13]:
def test_fn(model, dataloader):
    test_pred = np.ones(1)
    for inputs in tqdm(dataloader):
        with torch.no_grad():
            model = model.to(DEVICE) 
            pred = model(inputs)
            test_pred = np.concatenate( [ test_pred,
                                                         pred.cpu().detach().numpy().reshape(-1,) ] , axis=0)
    return test_pred[1:]

In [14]:
def predict_fold(n_fold, seed):
    test_loader = get_testloader(test_df, BATCH_SIZE=BATCH_SIZE)
    model = CLPmodel()
    model_path = f"{MODEL_PATH}/roberta-base-seed-{seed}-fold-{n_fold}.pth"
    model.load_state_dict(torch.load(model_path))
    model.to(DEVICE)

    prediction = test_fn(model, test_loader)

    del model
    gc.collect()
    return prediction

In [15]:
def predict_seed(seed):
    seed_everything(seed)
    pred = np.zeros( (submission.shape[0], ) )

    for n_fold in range(NUM_FOLDS):
        pred_fold = predict_fold(n_fold, seed)
        pred += pred_fold

    del pred_fold
    gc.collect()
    return pred / NUM_FOLDS

In [16]:
submission = test_df[['id']].copy()

DEVICE = 'cuda:0'
SEEDs = [0, 7 ,42, 88, 100]

BATCH_SIZE = 16
NUM_FOLDS = 5

MAX_WORDS = df["excerpt"].apply(lambda x: len(x.split())).max()

pred = np.zeros( (submission.shape[0], ) )
for seed in SEEDs:
    print(f'--------------- SEED {seed} is set ---------------')
    pred_seed = predict_seed(seed)
    pred += pred_seed
pred /= len(SEEDs)

submission['target'] = pred
submission.head()

--------------- SEED 0 is set ---------------


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  This is separate from the ipykernel package so we can avoid doing imports until



--------------- SEED 7 is set ---------------



--------------- SEED 42 is set ---------------



--------------- SEED 88 is set ---------------



--------------- SEED 100 is set ---------------


,id,target
0,c0f722661,-0.615447
1,f0953f0a5,-0.441347
2,0df072751,-0.592188
3,04caf4e0c,-2.380551
4,0e63f8bea,-1.829541


In [ ]:
submission.to_csv('submission.csv', index=False)